In [11]:
import json
from IPython.display import JSON

In [30]:
evaluation = []
with open('test_phi4_sample50_3x3.out.backup') as fd:
    line = fd.readline()
    while line:
        evaluation.append(json.loads(line))
        line = fd.readline()

In [31]:
len([i for i,ob in enumerate(evaluation) if 'Final answer:' in ob.get('full_prediction','')]) / (len(evaluation)-1)

0.0

In [32]:
[i for i,ob in enumerate(evaluation) if 'Final answer:' in ob.get('full_prediction','')]

[]

In [33]:
print(evaluation[1]['question'])

Who is the lead character in the Uncharted series?


1 correct ~
4 wrong
6 correct -
8 correct +
12 wrong
13 wrong
14 wrong

common problems:
- model answer too quickly --> prompt
- need to generate more triples --> prompt (maybe more than one-shot)
- need to generate different triples!! --> can I calculate number of leaves at index time to avoid the model to fall in the same leaf?

In [34]:
JSON(evaluation[8])

<IPython.core.display.JSON object>

In [26]:
i = 5
print(evaluation[i]['question'])
print(evaluation[i]['full_prediction'])

What city is the Arc de Triomphe in?
Reasoning: To answer this question, I need to identify the city where the Arc de Triomphe is located.
Fact: <Arc de Triomphe> <location> <Paris> .
Long answer: The Arc de Triomphe is a famous monument located in Paris, France. It stands at the western end of the Champs-Élysées and is an iconic symbol of the city.

Answer: Paris.<|im_end|>
